# Data Cleaning

In [11]:
import pandas as pd
!python -m pip install openpyxl


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\thema\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
df = pd.read_excel("../data/BSDS Class Times(1-28).xlsx")

In [13]:
df.columns
#Col to drop - "start time, completion time, email, name, last mod time, "

Index(['ID', 'Start time', 'Completion time', 'Email', 'Name',
       'Last modified time', 'What class are you?',
       'Do you prefer "blocks" of classes (e.g. two classes from 9:30 AM-12:15 PM) or stand alone classes (e.g. one class from 3:30-4:45 PM)',
       'Please rank which block class times you prefer',
       'Please rank which stand alone class times you prefer?',
       'Any comments or concerns?'],
      dtype='object')

### Getting the columns I need

In [14]:
df = df[['What class are you?',
       'Do you prefer "blocks" of classes (e.g. two classes from 9:30 AM-12:15 PM) or stand alone classes (e.g. one class from 3:30-4:45 PM)',
       'Please rank which block class times you prefer',
       'Please rank which stand alone class times you prefer?',
       'Any comments or concerns?']]

### Reformatting the ranking columns

In [15]:
df["preferences_list"] = (
    df["Please rank which block class times you prefer"]
    .str.strip(";")
    .str.split(";")
)

In [16]:
ranked = (
    df["preferences_list"]
    .apply(
        lambda prefs: {
            f"Block: {p}": i + 1
            for i, p in enumerate(prefs)
        }
    )
    .apply(pd.Series)
)

df = df.drop(columns="preferences_list").join(ranked)

In [17]:
df["preferences_list"] = (
    df["Please rank which stand alone class times you prefer?"]
    .str.strip(";")
    .str.split(";")
)

In [18]:
ranked = (
    df["preferences_list"]
    .apply(
        lambda prefs: {
            f"Stand Alone: {p}": i + 1
            for i, p in enumerate(prefs)
        }
    )
    .apply(pd.Series)
)

df = df.drop(columns="preferences_list").join(ranked)

In [19]:
df.drop(['Please rank which block class times you prefer',
       'Please rank which stand alone class times you prefer?'], axis = 1, inplace= True)
df = df.rename(columns={
    'Do you prefer "blocks" of classes (e.g. two classes from 9:30 AM-12:15 PM) or stand alone classes (e.g. one class from 3:30-4:45 PM)': "Blocks Or Stand Alone", 
    'What class are you?': 'Class'
})

In [20]:
df.to_csv("../data/data.csv", index = False)